好的，以下是“Understanding word2vec”部分的详细翻译：

---

### 理解word2vec（15分）

回想一下word2vec背后的关键见解是“通过周围的词语来认识一个词”。具体来说，考虑一个“中心”词 \(c\) ，它前后由一定长度的上下文包围。我们将这个上下文窗口中的词称为“外部词”（ \(O\) ）。例如，在图1中，上下文窗口的长度为2，中心词 \(c\) 是“banking”，外部词是“turning”、“into”、“crises”和“as”：

图1：带有窗口大小为2的word2vec skip-gram预测模型

Skip-gram word2vec旨在学习概率分布 \(P(O|C)\) 。具体来说，给定一个特定的词 \(o\) 和一个特定的词 \(c\) ，我们要预测 \(P(O = o|C = c)\) ：即词 \(o\) 是 \(c\) 的“外部”词的概率（即它落在 \(c\) 的上下文窗口内的概率）。我们通过对一系列向量点积使用softmax函数来建模这个概率：

\[ P(O = o | C = c) = \frac{\exp(u_o^\top v_c)}{\sum_{w \in \text{Vocab}} \exp(u_w^\top v_c)} \]

对于每个词，我们学习向量 \(u\) 和 \(v\) ，其中 \(u_o\) 是表示外部词 \(o\) 的“外部”向量， \(v_c\) 是表示中心词 \(c\) 的“中心”向量。我们将这些参数存储在两个矩阵 \(U\) 和 \(V\) 中。 \(U\) 的列是所有“外部”向量 \(u_w\) ； \(V\) 的列是所有“中心”向量 \(v_w\) 。 \(U\) 和 \(V\) 都包含词汇表中每个词的向量。

回忆一下讲座中，对于一对词 \(c\) 和 \(o\) ，损失函数为：

\[ J_{\text{naive-softmax}}(v_c, o, U) = -\log P(O = o|C = c) \]

我们可以将这个损失视为对于特定的中心词 \(c\) 和特定的外部词 \(o\) ，真实分布 \(y\) 和预测分布 \( \hat{y} \) 之间的交叉熵。这里， \(y\) 和 \( \hat{y} \) 都是长度等于词汇表中词数的向量。此外，这些向量中的第 \(k\) 个条目表示在给定 \(c\) 的情况下，第 \(k\) 个词是“外部词”的条件概率。真实经验分布 \(y\) 是一个one-hot向量，对于这个特定的中心词 \(c\) 和外部词 \(o\) ， \(o\) 的位置上为1，其他位置为0。预测分布 \( \hat{y} \) 是由我们模型在公式 (1) 中给出的概率分布 \(P(O|C = c)\) 。

注意：在整个作业中，当计算导数时，请使用讲座中复习的方法（即不使用泰勒级数近似）。

---

(a) (2分) 证明朴素softmax损失（公式2）与 \(y\) 和 \( \hat{y} \) 之间的交叉熵损失相同，即（注意 \(y\) （真实分布）， \( \hat{y} \) （预测分布）是向量， \( \hat{y}_o \) 是标量）：

\[ -\sum_{w \in \text{Vocab}} y_w \log(\hat{y}_w) = -\log(\hat{y}_o) \]

您的答案应该是一行。您可以用文字描述您的答案。

(b) (6分)

i. 计算 \(J_{\text{naive-softmax}}(v_c, o, U)\) 对 \(v_c\) 的偏导数。请用 \(y\) 、 \( \hat{y} \) 和 \(U\) 表示您的答案，并展示您的计算过程以获得全部分数。

- 注意：偏导数的最终答案应遵循形状约定：任何函数 \(f(x)\) 对 \(x\) 的偏导数应与 \(x\) 具有相同的形状。

- 请以矢量化形式提供偏导数的答案。例如，当我们要求您用 \(y\) 、 \( \hat{y} \) 和 \(U\) 表示答案时，您不能在最终答案中引用这些术语的具体元素（如 \(y_1\) 、 \(y_2\) 等）。

ii. 您计算的梯度何时为零？

提示：您可能希望复习并使用一些基础线性代数概念。

iii. 您找到的梯度是两个项之间的差。请解释这两个项在从词向量 \(v_c\) 中减去这个梯度时如何改善词向量。

(c) (1分) 在许多使用词嵌入的下游应用中，使用 \(L2\) 正则化向量（例如 \(u/||u||_2\) ，其中 \(||u||_2 = \sqrt{\sum u_i^2}\) ），而不是它们的原始形式（例如 \(u\) ）。让我们考虑一个假设的下游任务，即短语的二分类，判断其为正面还是负面，其中您根据单词的个别嵌入的总和来决定符号。 \(L2\) 正则化何时会带走下游任务的有用信息？何时不会？

提示：考虑 \(u_x = \alpha u_y\) 对于某些词 \(x \neq y\) 和某个标量 \( \alpha \) 。当 \( \alpha \) 为正时，正则化后的 \(u_x\) 和 \(u_y\) 的值是多少？此类正则化可能会如何影响或不影响结果分类？

(d) (5分) 计算 \(J_{\text{naive-softmax}}(v_c, o, U)\) 对每个“外部”词向量 \(u_w\) 的偏导数。将有两种情况：当 \(w = o\) 时，是真实的“外部”词向量，以及当 \(w \neq o\) 时，对于所有其他词。请用 \(y\) 、 \( \hat{y} \) 和 \(v_c\) 表示您的答案。在这个子部分中，您可以使用这些术语中的具体元素（如 \(y_1\) 、 \(y_2\) 等）。展示您的计算过程以获得全部分数。

(e) (1分) 写出 \(J_{\text{naive-softmax}}(v_c, o, U)\) 对 \(U\) 的偏导数。请将您的答案分解为列向量的形式 \(\frac{\partial J(v_c,o,U)}{\partial u_1}\) 、 \(\frac{\partial J(v_c,o,U)}{\partial u_2}\) 、···、 \(\frac{\partial J(v_c,o,U)}{\partial u_{|Vocab|}}\) 。不需要推导，只需给出矩阵形式的答案。

---

翻译完成了，如果还有其他部分需要翻译，请告知！